# Processing XMLs with ***lxml*** library

## Imports

In [14]:
import glob                  
import os
from lxml import etree as ET 
import zipfile
import csv
import logging
import shutil
import re

In [15]:
# Création du CSV qui contient les données des points de ventes
pdvcsvfile = open('pdv.csv', 'w')
pdvnames   = ["annee",
              "id",
              "latitude",
              "longitude",
              "cp",
              "pop",
              "ouverture_debut",
              "ouverture_fin",
              "ouverture_saufjour",
              "adresse",
              "ville"]
pdv_writer = csv.DictWriter(pdvcsvfile, fieldnames=pdvnames)
pdv_writer.writeheader()

# Création du CSV qui contient les données des ruptures
rupturescsvfile = open('ruptures.csv', 'w')
rupturesnames = ["annee",
                 "id_pdv",
                 "id",
                 "nom",
                 "debut",
                 "fin"]
ruptures_writer = csv.DictWriter(rupturescsvfile, fieldnames=rupturesnames)
ruptures_writer.writeheader()

# Création du CSV qui contient les données des fermetures
fermeturescsvfile = open('fermetures.csv', 'w')
fermeturesnames = ["annee",
                   "id_pdv",
                   "type",
                   "debut",
                   "fin"]
fermetures_writer = csv.DictWriter(fermeturescsvfile, fieldnames=fermeturesnames)
fermetures_writer.writeheader()


# Création du CSV qui contient les données des prix
prixcsvfile = open('prix.csv', 'w')
prixnames   = ["annee",
              "id_pdv",
              "nom",
              "id",
              "maj",
              "valeur"]
prix_writer = csv.DictWriter(prixcsvfile, fieldnames=prixnames)
prix_writer.writeheader()

# Création du CSV qui contient les données des services
servicescsvfile = open('services.csv', 'w')
servicesnames   = ["annee",
                   "id_pdv",
                   "service"]
services_writer = csv.DictWriter(servicescsvfile, fieldnames=servicesnames)
services_writer.writeheader()


# Initialisation des dictionnaire des lignes temporiaires à inserer dans les csv
pdv_row        = dict.fromkeys(pdvnames)
ruptures_row   = dict.fromkeys(rupturesnames)
fermetures_row = dict.fromkeys(fermeturesnames)
prix_row       = dict.fromkeys(prixnames)
services_row   = dict.fromkeys(servicesnames)

In [16]:
# txt_path : fonction qui prend une balise du XML (element), 
	# le chemin de la valeur qu’on souhaite récupérer (path) 
def txt_path(element, path='.'):
    if element.find(path) is not None:
        res = str(element.find(path).text)
        return res
    return ""

In [17]:
def process_prix(annee, id_pdv, prix):
    prix_row['annee']  = annee
    prix_row['id_pdv'] = id_pdv
    for name, value in sorted(prix.items()):
        prix_row[name] = value
    prix_writer.writerow(prix_row)

In [18]:
def process_service(annee, id_pdv, service):
    services_row['annee']   = annee
    services_row['id_pdv']  = id_pdv
    services_row["service"] = txt_path(service)
    services_writer.writerow(services_row)

In [19]:
def process_rupture(annee, id_pdv, rupture):
    ruptures_row['annee']  = annee
    ruptures_row['id_pdv'] = id_pdv
    for name, value in sorted(rupture.items()):
        ruptures_row[name] = value
    ruptures_writer.writerow(ruptures_row)

In [20]:
def process_fermeture(annee, id_pdv, fermeture):
    fermetures_row['annee']  = annee
    fermetures_row['id_pdv'] = id_pdv
    for name, value in sorted(fermeture.items()):
        fermetures_row[name] = value
    fermetures_writer.writerow(fermetures_row)

In [21]:
def process_pdv(annee, pdv):
    pdv_row['annee'] = annee
    pdv_row['adresse'] = txt_path(pdv, 'adresse')
    pdv_row['ville'] = txt_path(pdv, 'ville')
    
    for name, value in sorted(pdv.items()):
        pdv_row[name] = value
    
    for name, value in sorted(pdv.find('ouverture').items()):
        pdv_row["ouverture_" + name] = value
    
    id_pdv = pdv_row['id']
    for prix in pdv.findall('prix'):
        process_prix(annee, id_pdv, prix)
    
    for service in pdv.findall('services/service'):
        process_service(annee, id_pdv, service)
    
    for rupture in pdv.findall('rupture'):
        process_rupture(annee, id_pdv, rupture)
        
    for fermeture in pdv.findall('fermeture'):
        process_fermeture(annee, id_pdv, fermeture)
        
    pdv_writer.writerow(pdv_row)

In [13]:
for zip_file in glob.glob('data/PrixC*2019.zip'):
    annee   = re.findall('.*(\d{4})\.zip',zip_file)[0]
    xml     = re.findall('([^/]*\d{4})\.zip',zip_file)[0] + ".xml"

    # lire le zip
    logging.debug("Opening ZIP file " + zip_file)    
    zf = zipfile.ZipFile(zip_file, 'r')
    # lire le XML
    f  = zf.open(xml)
    # créer le XMLReader
    context = ET.iterparse(f, events=('end',), tag='pdv')
    for event, pdv in context:
        process_pdv(annee, pdv)
    # fermer le zip
    zf.close()

SAINT-DENIS-LèS-BOURG
BOURG-EN-BRESSE
Bourg-en-Bresse
SAINT-DENIS-LèS-BOURG
BOURG-EN-BRESSE
Bourg-en-Bresse
BOURG-EN-BRESSE
Bourg-en-Bresse
BOURG-EN-BRESSE
Saint-Denis-lès-Bourg
Bourg-en-Bresse
MONTCEAUX
Arbent
OYONNAX
Oyonnax
ARBENT - OYONNAX
Oyonnax
OYONNAX
HAUTEVILLE-LOMPNES
DAGNEUX
Montluel
MONTLUEL
LA BOISSE
La Boisse
DAGNEUX
DAGNEUX
Nantua
LE POIZAT LALLEYRIAT
NANTUA
Saint-Didier-sur-Chalaronne
LAGNIEU
SAINT-VULBAS
PONT-D'AIN
PONT-D'AIN
NEUVILLE-SUR-AIN
Pont-d'Ain
SéGNY
GEX
Gex
GEX
Saint-Bénigne
Bellegarde-sur-Valserine
Bellegarde-sur-Valserine
BELLEGARDE
FERNEY-VOLTAIRE
FERNEY-VOLTAIRE
FERNEY-VOLTAIRE
Divonne-les-Bains
SAINT-RAMBERT-EN-BUGEY
Torcieu
Jasseron
Ceyzériat
TOSSIAT
CEYZERIAT
COLIGNY
LAIZ
Belley
BELLEY
Chazey-Bons
POLLIAT
Confrançon
CHALAMONT
Chalamont
Ambérieux-en-Dombes
VILLARS-LES-DOMBES
Malafretaz
Jayat
ATTIGNAT
Montrevel-en-Bresse
CULOZ
Béon
BALAN
SAINT-ÉTIENNE-DU-BOIS
Saint-Étienne-du-Bois
Bâgé-la-Ville
Saint-André-de-Corcy
Saint-André-de-Corcy
MIONNAY
Mionnay
MI

Carcassonne
Carcassonne
CARCASSONNE
Carcassonne
Carcassonne
Carcassonne
Narbonne
CARCASSONNE
CARCASSONNE
CARCASSONNE
NARBONNE
NARBONNE
NARBONNE
NARBONNE
NARBONNE
Narbonne
Narbonne
Narbonne
NARBONNE
NARBONNE
NARBONNE
NARBONNE
NARBONNE
COURSAN
SALLES-D'AUDE
COURSAN
Salles-d'Aude
Salles-d'Aude
SALLES D'AUDE
Saint-Marcel-sur-Aude
Lézignan-Corbières
SIGEAN
AXAT
BRAM
VILLEPINTE
PEYRIAC-MINERVOIS
Villeneuve-Minervois
PEZENS
ALZONNE
LéZIGNAN-CORBIèRES
LéZIGNAN-CORBIèRES
Mauguio
LéZIGNAN-CORBIèRES
LéZIGNAN-CORBIèRES
LEZIGNAN CORBIERES
PORT-LA-NOUVELLE
PORT-LA-NOUVELLE
SAINT-LAURENT-DE-LA-CABRERISSE
Espéraza
ARZENS
ARZENS
Montréal
MONTREAL
Limoux
LIMOUX
Limoux
Limoux
LEUCATE
PORT LEUCATE
MOUTHOUMET
Castelnaudary
Castelnaudary
SAINT-MARTIN-LALANDE
Castelnaudary
CASTELNAUDARY
CASTELNAUDARY
GRUISSAN
GRUISSAN
LA PALME
La Palme
LA PALME
La Palme
La Palme
Quillan
Quillan
QUILLAN
ROQUEFORT-DES-CORBIèRES
Fleury
Sallèles-d'Aude
Sallèles-d'Aude
Villegly
Villalier
VILLEGLY
VILLEMOUSTAUSSOU
CAPENDU
Capendu


ROYAN
ROYAN
SAINT-SULPICE-DE-ROYAN
Royan
ROYAN
ROYAN
ROYAN
Bedenac
Bedenac
Chevanceaux
La Jarrie
Salles-sur-Mer
Saint-Rogatien
Charron
MARANS
Marans
SAINT-GENIS-DE-SAINTONGE
PONT-L'ABBE-D'ARNOULT
Saint-Porchaire
GEMOZAC
MONTGUYON
PUILBOREAU
AIGREFEUILLE-D'AUNIS
Aigrefeuille-d'Aunis
AIGREFEUILLE-D'AUNIS
ROCHEFORT
Rochefort
ROCHEFORT
ROCHEFORT
ROCHEFORT
ROCHEFORT
SAINT-PIERRE-D'OLERON
SAINT-PIERRE-D'OLéRON
SAINT-PIERRE-D'OLéRON
MARENNES
Marennes
Marennes
VILLENEUVE-LA-COMTESSE
Fenioux
Saint-Savinien
Saint-Savinien
FENIOUX
SAINT-AIGULIN
La Tremblade
SAINT-JEAN-D'ANGéLY
SAINT-JEAN-D'ANGéLY
Saint-Jean-d'Angély
SAINT-JEAN-D'ANGéLY
SAINT-MARTIN-DE-Ré
Saint-Martin-de-Ré
SAINT-PALAIS-SUR-MER
Tonnay-Charente
TONNAY-CHARENTE
Tonnay-Charente
AYTRE
AYTRé
FOURAS
Aulnay
Le Château-d'Oléron
JONZAC
JONZAC
Jonzac
ARVERT
ARVERT
DOLUS-D'OLéRON
BOURCEFRANC-LE-CHAPUS
LES MATHES
LE BOIS-PLAGE-EN-Ré
Ars-en-Ré
Saujon
MéDIS
Saujon
Medis
CHANIERS
Chaniers
ECHILLAIS
La Flotte
VAUX-SUR-MER
Vaux-sur-Mer
LA COUARDE-

Donzère
DONZERE
DONZERE
BOURG-DE-PéAGE
ALIXAN
Chatuzange-le-Goubet
Alixan
BOURG-DE-PéAGE
ALIXAN
Châteauneuf-sur-Isère
Chatuzange-le-Goubet
Bourg-de-Péage
ALIXAN
Châteauneuf-sur-Isère
Luc-en-Diois
SAINT-MARCEL-LèS-VALENCE
Saillans
PEYRINS
Hauterives
CREST
CREST
Crest
Aouste-sur-Sye
Crest
Crest
CREST
Cléon-d'Andran
Puy-Saint-Martin
Bourg-lès-Valence
BOURG-LèS-VALENCE
Bourg-lès-Valence
SAHUNE
Mours-Saint-Eusèbe
Pont-de-l'Isère
Tain-l'Hermitage
Tain-l'Hermitage
PONT-DE-L'ISèRE
PONT-DE-L'ISèRE
TAIN-L'HERMITAGE
Tain-l'Hermitage
Pont-de-l'Isère
PONT-DE-L'ISèRE
Pont-de-l'Isère
Pont-de-l'Isère
LUS-LA-CROIX-HAUTE
PIERRELATTE
Eymeux
La Baume-d'Hostun
La Baume-d'Hostun
La Baume-d'Hostun
LA BAUME D'HOSTUN
LA BAUME D'HOSTUN
La Baume-d'Hostun
La Baume-d'Hostun
LA BAUME D'HOSTUN
MONTBOUCHER-SUR-JABRON
MONTBOUCHER-SUR-JABRON
LES TOURRETTES
MONTBOUCHER SUR JABRON
Saint-Paul-lès-Romans
Beaumont-lès-Valence
ALLAN
CHATEAUNEUF DU RHONE
ALLAN
MALATAVERNE
Châteauneuf-du-Rhône
Châteauneuf-du-Rhône
TULETTE
Tule

BORDEAUX
BORDEAUX
BORDEAUX
BORDEAUX
BORDEAUX
BORDEAUX
BORDEAUX
BORDEAUX
BORDEAUX
LE BOUSCAT
LE BOUSCAT
Le Bouscat
Le Bouscat
LE BOUSCAT
Le Bouscat
SAINT-LAURENT-MéDOC
SAINT-SYMPHORIEN
LE BARP
ARCACHON
ARCACHON
Arcachon
Arcachon
ARCACHON
CARCANS
AILLAS
Saint-Jean-d'Illac
Saint-Jean-d'Illac
MARTIGNAS-SUR-JALLE
BèGLES
BEGLES
Bègles
BèGLES
Bègles
GALGON
Lanton
VILLENAVE-D'ORNON
VILLENAVE-D'ORNON
Villenave-d'Ornon
Cadaujac
CADAUJAC
VILLENAVE D'ORNON
CADAUJAC
Cenon
CENON
SAINT-MéDARD-EN-JALLES
Saint-Médard-en-Jalles
GRADIGNAN
GRADIGNAN
GRADIGNAN
Gradignan
GRADIGNAN
GRADIGNAN
LE HAILLAN
LE HAILLAN
GIRONDE-SUR-DROPT
La Réole
Gironde-sur-Dropt
BORDEAUX CAUDERAN
LANGON
LANGON
Langon
LANGON
LANGON
LANGON
PORT-SAINTE-FOY-ET-PONCHAPT
Pineuilh
PORT-SAINTE-FOY-ET-PONCHAPT
PINEUILH
PINEUILH
COUTRAS
COUTRAS
SAINT-ANDRé-DE-CUBZAC
SAINT-ANDRé-DE-CUBZAC
CADILLAC-EN-FRONSADAIS
SAINT-ANDRé-DE-CUBZAC
Cubzac-les-Ponts
SAINT-ANDRé-DE-CUBZAC
LA LANDE-DE-FRONSAC
ST ANDRE DE CUBZAC
Saint-André-de-Cubzac
PAUILLAC


SAINT-PIERRE-DES-CORPS
SAINT-PIERRE-DES-CORPS
La Ville-aux-Dames
SAINT PIERRE DES CORPS
ST PIERRE DES CORPS
Saint-Pierre-des-Corps
SAINTE-MAURE-DE-TOURAINE
Saint-Épain
SAINT-ÉPAIN
Noyant-de-Touraine
SAINTE-MAURE DE TOURAINE
Saint-Épain
SAINT EPAIN
Saint-Épain
GRENOBLE
Grenoble
GRENOBLE
Grenoble
Grenoble
GRENOBLE
SAINT-QUENTIN-FALLAVIER
L'Isle-d'Abeau
L'ISLE-D'ABEAU
L'ISLE D'ABEAU
L'Isle-d'Abeau
L'Isle-d'Abeau
L'Isle-d'Abeau
VILLEFONTAINE
VILLEFONTAINE
Villefontaine
VILLEFONTAINE
GRENOBLE
Grenoble
Grenoble
Grenoble
GRENOBLE
GRENOBLE
SAINT-CLAIR-DE-LA-TOUR
La Tour-du-Pin
SAINT-JEAN-DE-SOUDAIN
VEUREY-VOROIZE
SAINT-ÉGRèVE
Saint-Égrève
Reventin-Vaugris
Échirolles
RIVES
APPRIEU
Le Freney-d'Oisans
SALAISE-SUR-SANNE
Chanas
ROUSSILLON
CHANAS
SALAISE-SUR-SANNE
Salaise-sur-Sanne
SALAISE-SUR-SANNE
Salaise-sur-Sanne
SAINT-ROMANS
Chatte
SAINT-SAUVEUR
SAINT-MARCELLIN
Saint-Marcellin
SEYSSINET-PARISET
SEYSSINET-PARISET
SEYSSINET-PARISET
SEYSSINET
Seyssins
Froges
Saint-Nazaire-les-Eymes
VIENNE
Vienne
V

CARQUEFOU
DONGES
LE CROISIC
La Baule-Escoublac
La Baule-Escoublac
LA BAULE-ESCOUBLAC
LE POULIGUEN
MéSANGER
MESANGER
Saint-Gildas-des-Bois
SAINT-MARS-LA-JAILLE
MONTOIR-DE-BRETAGNE
PAIMBOEUF
Trignac
TRIGNAC
TRIGNAC
TRIGNAC
TRIGNAC
Bourgneuf-en-Retz
BOURGNEUF-EN-RETZ
DERVAL
Derval
SAINT-NAZAIRE
SAINT-NAZAIRE
SAINT-NAZAIRE
SAINT-NAZAIRE
SAINT-NAZAIRE
SAINT-NAZAIRE
SAINT-NAZAIRE
SAINT NAZAIRE
Saint-Nazaire
La Montagne
Plessé
LE PELLERIN
LEGE
SAINTE-PAZANNE
Orvault
Orvault
Orvault
ORVAULT
PORT SAINT-PERE
Saint-Joachim
Saint-Joachim
SAINT MICHEL CHEF CHEF
SAINT-MICHEL-CHEF-CHEF
Batz-sur-Mer
CAMPBON
LA PLAINE-SUR-MER
SAINT-HERBLAIN
SAINT-HERBLAIN
SAINT-HERBLAIN
Saint-Herblain
SAINT-HERBLAIN
Saint-Herblain
Saint-Herblain
SAINT-HERBLAIN
Héric
BOUAYE
Les Sorinières
LES SORINIERES
LIGNE
SAINT-MARS-DU-DéSERT
PONT-SAINT-MARTIN
SAUTRON
Sautron
ORLéANS
ORLEANS
ORLEANS
ORLEANS
ORLEANS
Orléans
Orléans
ORLéANS
ORLEANS
Orléans
Châteauneuf-sur-Loire
Châteauneuf-sur-Loire
CHâLETTE-SUR-LOING
CHâLETTE-SUR-LOI

LE BOURGNEUF-LA-FORêT
LA GRAVELLE
Saint-Pierre-la-Cour
Vaiges
ERNEE
ERNéE
ERNEE
ÉVRON
ÉVRON
ÉVRON
Évron
Evron
Villaines-la-Juhel
RENAZE
Changé
LAVAL
LOUVERNé
Bonchamp-lès-Laval
Bonchamp-lès-Laval
BONCHAMP-LèS-LAVAL
L'HUISSERIE
Nancy
NANCY
NANCY
NANCY
NANCY
Nancy
Nancy
NANCY
NANCY
DOMBASLE-SUR-MEURTHE
Dombasle-sur-Meurthe
Baccarat
DENEUVRE
VANDIèRES
Saint-Max
Saint-Max
Ceintrey
MEXY
Jarville-la-Malgrange
BRIEY
BRIEY
LANTéFONTAINE
COLOMBEY-LES-BELLES
Colombey-les-Belles
Colombey-les-Belles
Colombey-les-Belles
HEILLECOURT
TOUL
Dommartin-lès-Toul
TOUL
TOUL
Écrouves
CHAUDENEY
CHAUDENEY
TOUL
DOMMARTIN-LèS-TOUL
Saint-Nicolas-de-Port
Saint-Nicolas-de-Port
Malzéville
Neuves-Maisons
NEUVES-MAISONS
JOEUF
Joeuf
CHAMPIGNEULLES
TELLANCOURT
SAINT-JEAN-LèS-LONGUYON
LONGUYON
ESSEY-LèS-NANCY
Essey-lès-Nancy
SEICHAMPS
SEICHAMPS
BAYON
Moncel-lès-Lunéville
LUNéVILLE
LUNéVILLE
VITRIMONT
VITRIMONT
LUNEVILLE
Chanteheux
LUNEVILLE
BLAINVILLE SUR L'EAU
VITRIMONT
LUNEVILLE
Homécourt
Maxéville
VÉZELISE
POMPEY
DAME

KeyboardInterrupt: 

In [15]:
pdvcsvfile.close()
rupturescsvfile.close()
fermeturescsvfile.close()
servicescsvfile.close()
prixcsvfile.close()